# FX AI Backtest и метрики

Этот ноутбук строит простой бэктест на основе уже обученных моделей внутри `app.py`.

Подход:
- используем ту же логику подготовки данных и обучения, что и в Streamlit-приложении;
- берём предсказания классификатора (по умолчанию SVC) на тестовом отрезке;
- строим стратегию: позиция = сигнал (BUY=+1, HOLD=0, SELL=-1), удерживаем один шаг горизонта вперёд;
- считаем дневную доходность стратегии, кривую капитала, Sharpe, max drawdown, hit rate;
- сравниваем с простым buy&hold на том же периоде.

Такой бэктест намеренно упрощён (нет комиссий, проскальзываний и т.п.), но позволяет быстро оценить поведение сигналов.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import plotly.graph_objects as go

sys.path.append(os.path.abspath("."))

from app import (
    load_price_data,
    add_features,
    add_targets,
    train_models,
    detect_patterns,
)

## 1. Конфигурация инструмента

Задаём базовые параметры для бэктеста. При желании их можно менять и запускать ноутбук повторно.

In [ ]:
ticker = "EURUSD=X"
instrument_name = "EUR/USD"
years = 5
interval = "1d"

horizon = 7
lower_q = 0.33
upper_q = 0.66

config = {
    "ticker": ticker,
    "instrument_name": instrument_name,
    "years": years,
    "interval": interval,
    "horizon": horizon,
    "lower_q": lower_q,
    "upper_q": upper_q,
}
config

## 2. Подготовка данных и обучение моделей

Повторяем тот же пайплайн, что и внутри `app.py`: загрузка данных, фичи, таргеты, обучение моделей.
Нас интересует в первую очередь классификационная часть и разбиение на train/test.

In [ ]:
df_raw = load_price_data(ticker, years=years, interval=interval)
if df_raw is None or df_raw.empty:
    raise RuntimeError("Не удалось загрузить данные по тикеру")

df_full = add_features(df_raw)
df_model = add_targets(
    df_full.copy(),
    horizon=horizon,
    lower_q=lower_q,
    upper_q=upper_q,
)
model_data = train_models(df_model)
split = model_data["split"]
metrics = model_data.get("metrics")
df_model.tail()

## 3. Построение простой стратегии на основе сигналов SVC

Берём предсказания SVC по классам:
- `-1` — SELL,
- `0` — HOLD,
- `1` — BUY.

Используем тестовый отрезок (последние 20% данных), на каждом шаге держим позицию, равную сигналу, и считаем доходность на один шаг вперёд.

In [ ]:
df_back = df_model.iloc[split:].copy()
svc_pred = model_data.get("svc_pred_test")
if svc_pred is None:
    raise RuntimeError("В model_data нет предсказаний SVC")

svc_pred = np.asarray(svc_pred, dtype=float)
n = min(len(df_back), len(svc_pred))
df_back = df_back.iloc[-n:].copy()
signals_svc = svc_pred[-n:]

df_back["signal_svc"] = signals_svc

close = df_back["Close"].astype(float)
next_ret = close.pct_change().shift(-1)
df_back["next_ret"] = next_ret
df_back["strategy_ret_svc"] = df_back["signal_svc"] * df_back["next_ret"]

df_back[["Close", "signal_svc", "next_ret", "strategy_ret_svc"]].tail(10)

## 4. Подсчёт метрик: доходность, Sharpe, max drawdown, hit rate

Считаем:
- накопленную доходность стратегии и buy&hold;
- дневной Sharpe (≈ sqrt(252) * mean / std);
- максимальную просадку по кривой капитала;
- hit rate (доля дней, когда стратегия правильно угадывает знак следующего движения).

In [ ]:
ret_strat = df_back["strategy_ret_svc"].fillna(0.0)
equity_strat = (1.0 + ret_strat).cumprod()

equity_bh = close / close.iloc[0]

cum_ret_strat = float(equity_strat.iloc[-1] - 1.0)
cum_ret_bh = float(equity_bh.iloc[-1] - 1.0)

mean_daily = float(ret_strat.mean())
std_daily = float(ret_strat.std())
if std_daily > 0:
    sharpe = float(np.sqrt(252.0) * mean_daily / std_daily)
else:
    sharpe = 0.0

running_max = equity_strat.cummax()
drawdown = equity_strat / running_max - 1.0
max_dd = float(drawdown.min())

mask_valid = df_back["next_ret"].notna() & (df_back["signal_svc"] != 0)
hits = np.sign(df_back.loc[mask_valid, "signal_svc"]) == np.sign(df_back.loc[mask_valid, "next_ret"])
if len(hits) > 0:
    hit_rate = float(hits.mean())
else:
    hit_rate = 0.0

metrics_table = pd.DataFrame(
    [
        {
            "Метрика": "Cumulative return (strategy)",
            "Значение": cum_ret_strat,
        },
        {
            "Метрика": "Cumulative return (buy&hold)",
            "Значение": cum_ret_bh,
        },
        {
            "Метрика": "Sharpe (strategy, daily)",
            "Значение": sharpe,
        },
        {
            "Метрика": "Max drawdown (strategy)",
            "Значение": max_dd,
        },
        {
            "Метрика": "Hit rate (direction, strategy)",
            "Значение": hit_rate,
        },
    ]
)
metrics_table

## 5. Добавляем стратегии по сигналам LGBM и Hybrid

В `model_data` уже есть предсказания других классификаторов:
- `class_pred_test` — сигналы LGBM (после порога и сглаживания);
- `hybrid_class_pred_test` — гибрид LGBM+LSTM (если LSTM доступен).

Построим для них такие же простые стратегии, как для SVC, и сравним метрики.

In [ ]:
def build_strategy_from_signals(df_base: pd.DataFrame, signals_raw: np.ndarray, column_name: str):
    if signals_raw is None:
        return None
    sig = np.asarray(signals_raw, dtype=float)
    n_local = min(len(df_base), len(sig))
    df_loc = df_base.iloc[-n_local:].copy()
    df_loc[column_name] = sig[-n_local:]
    close_loc = df_loc["Close"].astype(float)
    next_ret_loc = close_loc.pct_change().shift(-1)
    df_loc[f"{column_name}_ret"] = df_loc[column_name] * next_ret_loc
    return df_loc

lgbm_pred = model_data.get("class_pred_test")
hybrid_pred = model_data.get("hybrid_class_pred_test")

df_lgbm = build_strategy_from_signals(df_back, lgbm_pred, "signal_lgbm")
df_hybrid = build_strategy_from_signals(df_back, hybrid_pred, "signal_hybrid")

strategies_equity = {"svc": equity_strat}
if df_lgbm is not None:
    ret_lgbm = df_lgbm["signal_lgbm_ret"].fillna(0.0)
    strategies_equity["lgbm"] = (1.0 + ret_lgbm).cumprod()
if df_hybrid is not None:
    ret_hybrid = df_hybrid["signal_hybrid_ret"].fillna(0.0)
    strategies_equity["hybrid"] = (1.0 + ret_hybrid).cumprod()

metrics_rows = []
for name, eq in strategies_equity.items():
    ret_seq = eq.pct_change().fillna(0.0)
    cum_ret = float(eq.iloc[-1] - 1.0)
    mean_d = float(ret_seq.mean())
    std_d = float(ret_seq.std())
    sharpe_local = float(np.sqrt(252.0) * mean_d / std_d) if std_d > 0 else 0.0
    run_max = eq.cummax()
    dd = eq / run_max - 1.0
    max_dd_local = float(dd.min())
    metrics_rows.append(
        {
            "Стратегия": name,
            "Cumulative return": cum_ret,
            "Sharpe": sharpe_local,
            "Max drawdown": max_dd_local,
        }
    )

metrics_multi = pd.DataFrame(metrics_rows)
metrics_multi

## 6. Визуализация PnL стратегий SVC / LGBM / Hybrid vs buy&hold

Теперь сравним сразу несколько стратегий на одном графике: SVC, LGBM, Hybrid и buy&hold.

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=equity_strat.index,
        y=equity_strat.values,
        mode="lines",
        name="Strategy (SVC)",
        line=dict(color="blue", width=2),
    )
)
if "lgbm" in strategies_equity:
    eq_lgbm = strategies_equity["lgbm"]
    fig.add_trace(
        go.Scatter(
            x=eq_lgbm.index,
            y=eq_lgbm.values,
            mode="lines",
            name="Strategy (LGBM)",
            line=dict(color="green", width=2, dash="dot"),
        )
    )
if "hybrid" in strategies_equity:
    eq_hybrid = strategies_equity["hybrid"]
    fig.add_trace(
        go.Scatter(
            x=eq_hybrid.index,
            y=eq_hybrid.values,
            mode="lines",
            name="Strategy (Hybrid)",
            line=dict(color="orange", width=2, dash="dash"),
        )
    )
fig.add_trace(
    go.Scatter(
        x=equity_bh.index,
        y=equity_bh.values,
        mode="lines",
        name="Buy & Hold",
        line=dict(color="gray", width=2, dash="dashdot"),
    )
)
fig.update_layout(
    title=f"PnL стратегий (SVC / LGBM / Hybrid) vs Buy&Hold ({instrument_name}, test segment)",
    xaxis_title="Дата",
    yaxis_title="Капитал (нормирован)",
    hovermode="x unified",
    template="plotly_white",
    legend=dict(orientation="h"),
)
fig.show()

## 7. Рекомендации и типичные ошибки такого бэктеста

1. **Отсутствие транзакционных издержек**  
   В текущем расчёте нет комиссий, спреда и проскальзывания. В реальности FX-спред и комиссии брокера могут существенно уменьшить доходность, особенно если стратегия часто торгует.

2. **Фиксированный горизонт удержания**  
   Здесь предполагается, что сигнал действует ровно один шаг вперёд. В реальности сигналы могут иметь разный "срок жизни", и может потребоваться более сложная логика выхода (take profit, stop loss, трейлинг и т.п.).

3. **Использование только одного классификатора (SVC)**  
   В проекте есть несколько моделей (LGBM, LSTM, Hybrid). Этот ноутбук использует только SVC как основную модель для сигналов. Для более полного анализа стоит построить такие же бэктесты для разных вариантов сигналов и сравнить результаты.

4. **Риск подгонки к истории**  
   Параметры моделей, горизонта и порогов классификации подбирались на истории. При изменении режима рынка качество может ухудшиться. Полезно:
   - проверять модель на нескольких несоприкасающихся временных отрезках;
   - регулярно переобучать модели и отслеживать скользящие метрики.

5. **Частота сигналов и фильтрация по волатильности**  
   Здесь в стратегию попадают все сигналы SVC. В проекте уже есть оценка волатильности по ATR(14); логичным развитием будет:
   - не торговать при экстремально высокой волатильности (снижать размер позиции или полностью фильтровать сигналы);
   - адаптировать порог уверенности классификатора в зависимости от волатильности.

6. **Неявная зависимость от конкретного периода теста**  
   Мы используем последнее 20% окно данных как тест. Если рынок в этот период был аномальным, метрики могут быть не репрезентативны. Для более надёжной оценки стоит реализовать скользящее или перекрёстное тестирование (например, walk-forward).

Этот ноутбук даёт отправную точку для оценки качества сигналов на исторических данных. Дальше можно наращивать сложность: добавлять комиссии, тестировать разные модели/пороговые стратегии и строить отчёты по множеству тикеров.